In [ ]:
import math
import operator
import matplotlib.pyplot as plt
#python在读写matlab文件时常用到scipy.io文件
import scipy.io as sio
import openpyxl
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,BatchNormalization,LeakyReLU,Conv2DTranspose,Reshape,Conv2D,Dropout,Flatten,AveragePooling1D,Activation
import tensorflow as tf
import time
import os
import matplotlib
from keras.models import Model
import keras
from scipy.signal import savgol_filter
from numpy import *
os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
x = np.load('trainx.npy',allow_pickle = True)
y = np.load('trainy.npy',allow_pickle = True)
#定义真实光谱数据
wave = x.reshape(-1,2500)
wave = wave.astype('float64')
print(wave.shape)

In [ ]:
P = np.load('axis_x.npy',allow_pickle =True)

In [ ]:
#绘制频谱
def PlotSpectrum(spec,axis_x):
#创建画布
    plt.figure(figsize=(5.2, 3.1), dpi=300)
#np.arange()属于numpy模块。
#生成一个从start（包含）到stop（不包含），以step为步长的序列。返回一个ndarray对象
#spec.shape[1]为spec的第二个维度大小
#np.arange()
#1）一个参数时，参数值为终点，起点取默认值0，步长取默认值1。
#2）两个参数时，第一个参数为起点，第二个参数为终点，步长取默认值1。
#3）三个参数时，第一个参数为起点，第二个参数为终点，第三个参数为步长。其中步长支持小数
    x = axis_x
    for i in range(spec.shape[0]):
        #取spec的第i行全部列作为y值
        plt.plot(x, spec[i, :], linewidth=0.6)
    fonts = 8
    plt.xlim(x.min(),x.max())
    plt.ylim(0, 1)
    # 设置刻度字体大小
    plt.xlabel('Wavelength (nm)', fontsize=fonts)
    plt.ylabel('absorbance (AU)', fontsize=fonts)
    plt.yticks(fontsize=fonts)
    plt.xticks(fontsize=fonts)
    #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
    #调整方格
    plt.tight_layout(pad=0.3)
    #网格线设置
    plt.grid(True)
    return plt
PlotSpectrum(wave,P)

In [ ]:
#定义一些常数
# Dataset中的buffer
pre_buffer_size = 60
# 批次大小（530/batch_size=程序要跑的次数）
pre_batch_size = 15
buffer_size = 60
batch_size = 30

In [ ]:
# 训练周期
epochs = 6500
# 100维的随机噪声
noise_dim = 200
# 除开理化值的噪声维度
noise_dim_0 = 500

In [ ]:
#k的比例增益初始化为lambda_k=0.0001,用于不断调整k的值
#经过20000次训练变成0.00001
lambda_k1=0.0002
lambda_k2=0.00001
lambda_k3=0.000001
#k的初值设为：0
k = tf.Variable(0.005,dtype=tf.float64)
#设置gamma(高多样性好)
gamma = tf.Variable(0.7,dtype=tf.float64)

In [ ]:
def generator_model():
    #顺序模型设置
    model = tf.keras.Sequential()
    #传入噪声数据
    model.add(Dense(1000,input_shape=(noise_dim_0,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1800))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(2500))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    return model

In [ ]:
# from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model

generator = generator_model()
# generator = multi_gpu_model(generator,gpus=2)
generator.summary()

In [ ]:
#定义判别器
def discriminator_model():
    #顺序模型
    model = tf.keras.Sequential()
    model.add(Dense(1800,input_shape=(2500,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1000))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(500))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1000))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1800))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(2500))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    return model

In [ ]:
discriminator = discriminator_model()
discriminator.summary()

In [ ]:
discriminator.get_layer(index=11).output

In [ ]:
# layer_name = 'leaky_re_lu_11' #获取层的名称
noiseTest = tf.random.normal([batch_size, noise_dim_0])
# generator1 = Model(inputs=generator.input, 
#                                  outputs=generator.get_layer(index=2).output)# 输出为1800

discriminator1 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=2).output)# 输出为1800，第2层

discriminator2 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=5).output)# 输出为1800，第8层

discriminator3 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=11).output)# 输出为1800，第8层

discriminator4 = Model(inputs=discriminator.input, 
                                 outputs=discriminator.get_layer(index=14).output)# 输出为1800，第8层

# intermediate_output = intermediate_layer_model.predict(noise)


In [ ]:
discriminator1(wave)

In [ ]:
discriminator2(wave)

In [ ]:
discriminator3(wave)

In [ ]:
discriminator4(wave)

In [ ]:
# MSG-GAN多尺度梯度生成式对抗网络
def L1(wave, D1, D2):# D1:wave的编码部分；D2：wave的解码部分
    #通过判别器得到D_ture
    D_output1 = D1(wave)# D1:1800
    D_output2 = D2(wave)# D2:1800
    D_output1 = tf.cast(D_output1,dtype=tf.float64)
    D_output2 = tf.cast(D_output2,dtype=tf.float64)
    #BEGAN的D的损失函数的前半部分(L(x))
    L1_loss = tf.reduce_mean(tf.abs(D_output1 - D_output2))
    L1_loss = tf.cast(L1_loss,dtype=tf.float64)
    return L1_loss 
def L2(noise, G, D1, D2):# D1:wave的编码部分；D2：wave的解码部分
    G_output = G(noise)# G:2500
    G_output = tf.cast(G_output,dtype=tf.float64)    
    D_output1 = D1(G_output)# D1:1800
    D_output1 = tf.cast(D_output1,dtype=tf.float64)
    D_output2 = D2(G_output)# D2:1800
    D_output2 = tf.cast(D_output2,dtype=tf.float64)
    #作差-取绝对-求平均值(L(G(x)))
    L2_loss = tf.reduce_mean(tf.abs(D_output2 - D_output1))
    L2_loss = tf.cast(L2_loss,dtype=tf.float64)
    return L2_loss 
def L3(wave, D):
    #通过判别器得到D_ture
    D_output = D(wave)
    D_output = tf.cast(D_output,dtype=tf.float64)
    #BEGAN的D的损失函数的前半部分(L(x))
    L3_loss = tf.reduce_mean(tf.abs(D_output - wave))
    L3_loss = tf.cast(L3_loss,dtype=tf.float64)
    return L3_loss 
def L4(noise, G, D):# D1:wave的编码部分；D2：wave的解码部分
    G_output = G(noise)# G:完整的generator
    G_output = tf.cast(G_output,dtype=tf.float64)    
    D_output = D(G_output)# 编码到指定尺寸
    D_output = tf.cast(D_output,dtype=tf.float64)
    #作差-取绝对-求平均值(L(G(x)))
    L4_loss = tf.reduce_mean(tf.abs(G_output - D_output))
    L4_loss = tf.cast(L4_loss,dtype=tf.float64)
    return L4_loss 
#定义生成器损失
def LG(noise, G1, D1, D2):# D: 适配G尺寸的自编码器
    G_loss = L2(noise, G1, D1, D2)
    return G_loss
# 
def LG1(noise, G, D):# 原始损失函数
    G_loss = L4(noise, G, D)
    return G_loss
def LD(wave, noise, k, G, D1, D2):# D1:wave的编码部分；D2：wave的解码部分；
    loss1 = L1(wave, D1, D2)
    loss2 = L2(noise, G, D1, D2)
    total_loss = loss1 - k*loss2
    return total_loss
# 
def LD1(wave, noise, k, G, D):# 原始损失函数
    loss1 = L3(wave, D)
    loss2 = L4(noise, G, D)
    total_loss = loss1 - k*loss2
    return total_loss
#更新k参数
def update_k():
    global k
    k = tf.compat.v1.assign(k, tf.clip_by_value(k + lambda_k * balance, 0, 1))
#定义损失函数
#预定义L(G(x))
def L_G(noise):
    #通过生成器生成的值G_fake
    G_fake = generator(noise)
    G_fake = tf.cast(G_fake,dtype=tf.float64)
    #通过判别器得到的结果D_fake
    D_fake = discriminator(G_fake)
    D_fake = tf.cast(D_fake,dtype=tf.float64)
    #作差-取绝对-求平均值(L(G(x)))
    L_G = tf.reduce_mean(tf.abs(G_fake-D_fake))
    return L_G
#预定义L(x)
def L(ture_wave):
    #通过判别器得到D_ture
    D_ture = discriminator(ture_wave)
    D_ture = tf.cast(D_ture,dtype=tf.float64)
    #BEGAN的D的损失函数的前半部分(L(x))
    L_x = tf.reduce_mean(tf.abs(ture_wave-D_ture))
    return L_x   
#定义生成器损失
def G_LOSS(noise):
    G_loss = L_G(noise)
    return G_loss
def D_LOSS(ture_wave,noise,k):
    L1 = L(ture_wave)
    LG1 = L_G(noise)
    total_loss = L1 - k*LG1
    return total_loss
#更新k参数
def update_k():
    global k
    # print('k值为：%s'%k)
    k = tf.compat.v1.assign(k, tf.clip_by_value(k + lambda_k * balance, 0, 1))
#定义总体进度函数
def global_pro(ture_wave,noise,gamma):
    L2 = L(ture_wave)
    LG2 = L_G(noise)
    balance = gamma*L2 - LG2
    balance = tf.cast(balance,dtype=tf.float64)
    #总体进度定义
    global_goal = L2 + tf.abs(balance)
    return global_goal,balance

In [ ]:
#分别定义两个网络，判别器和生成器优化器
generator_optimizer = tf.keras.optimizers.Adam(0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)
#切分数据集
pre_train_dataset = tf.data.Dataset.from_tensor_slices(wave).shuffle(pre_buffer_size).batch(pre_batch_size)
train_dataset = tf.data.Dataset.from_tensor_slices(wave).shuffle(buffer_size).batch(batch_size)

In [ ]:
#提前训练D
#定义提前训练损失函数
def pretrainLoss(wave, D):
    #损失函数就是L(x)
    treated_D = L(wave)
    return treated_D
#需要记录训练损失的变化
Epochs = 200
def Pre_train_D(dataset,Epochs):
    summary_writer = tf.summary.create_file_writer('D:/Pre_train')
    # 训练epochs周期
    for epoch in range(Epochs):
        # 每次获得一个批次的真实图片传入train_step函数进行训练
        for wave_batch in dataset:
            with tf.GradientTape() as DISC_tape:
                #计算判别器LOSS（只输入真实数据）
                disc_loss = pretrainLoss(wave_batch, discriminator)
            # 传入loss和模型参数，计算判别器的权值调整
            gradients_of_discriminator = DISC_tape.gradient(disc_loss, discriminator.trainable_variables)
            # 判别器的权值调整
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        with summary_writer.as_default():
            tf.summary.scalar('loss',disc_loss,step=epoch)
        if epoch%20 ==0:
            print(disc_loss)

In [ ]:
Start = time.perf_counter()
Pre_train_D(pre_train_dataset,Epochs)
End = time.perf_counter()
print('Running time: %s Seconds'%(End-Start))

In [ ]:
generator.save('MSG_pregeneratorLast.h5')
discriminator.save('MSG_prediscriminatorLast.h5')

In [ ]:
p = P
# 生成图片并保存显示
def generate_and_save_images(model, test_input):
    # 注意training设定为False，所有层都在预测模式下运行
    predictions = model(test_input, training=False)
    # 显示图片
    #建立横轴
    #设置点
    predictions = np.array(predictions)
    predictions = predictions.reshape(120,x.shape[1])
    for i in range(predictions.shape[0]):
        #取spec的第i行全部列作为y值
        plt.plot(p, predictions[i, :], linewidth=0.6)
    fonts = 8
    fonts = 8
    plt.xlim(p.min(),p.max())
    # 设置刻度字体大小
    plt.xlabel('Wavelength (nm)', fontsize=fonts)
    plt.ylabel('Absorbance (AU)', fontsize=fonts)
    plt.yticks(fontsize=fonts)
    plt.xticks(fontsize=fonts)
    #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
    # 显示图片
    plt.show()
def generate_and_save_images_b(model, i, test_input):
    # 注意training设定为False，所有层都在预测模式下运行
    predictions = model(test_input, training=False)
    # 显示图片
    #建立横轴
    #设置点
    predictions = np.array(predictions)
    predictions = predictions.reshape(x.shape[1],1)
    fonts = 8
    plt.xlim(p.min(),p.max())
    # 设置刻度字体大小
    plt.xlabel('Wavelength (nm)', fontsize=fonts)
    plt.ylabel('Absorbance (AU)', fontsize=fonts)
    plt.yticks(fontsize=fonts)
    plt.xticks(fontsize=fonts)
    #Padding between the figure edge and the edges of subplots, as a fraction of the font size.
    #调整方格
    plt.tight_layout(pad=0.3)
    #网格线设置
    plt.plot(P,predictions,linewidth=0.5,c = 'k')
    # 显示图片
    plt.show()

In [ ]:
# 训练模型
process = []
def train_0(dataset, epochs):
    # 训练epochs周期
    #将k值变化放到K文件中
    summary_K = tf.summary.create_file_writer('D:/K')
    #将dis_loss和gen_loss放在LOSS中
    summary_LOSS = tf.summary.create_file_writer('D:/LOSS')
    #定义全局k
    global k
    goal_container = []
    num_patience = 0
    for epoch in range(epochs):
        # 每次获得一个批次的真实图片传入train_step函数进行训练
        #这里应该根据传入的epoch，设置一个梯度的训练参数
        #当epoch小于2000次时，使用
        if epoch<=500:
            for wave_batch in dataset:
                # 生成一个批次的随机数，这个随机数用于模型训练
                noise = tf.random.normal([batch_size, noise_dim_0])   #输出goal全局得分
                goal,balance = global_pro(wave_batch, noise, gamma)
                #输出goal
                print('全局得分：%s'%goal)
                #输出全局k
                print('k:%s'%k)
                goal = float(goal)
                # print(goal)
                goal_container.append(goal)
                # 固定写法，使用tf.GradientTape()来计算梯度
                with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                    # 计算生成器loss
                    gen_loss1 = LG1(noise, generator, discriminator)
                    gen_loss2 = LG(noise, generator, discriminator1, discriminator4)
                    gen_loss3 = LG(noise, generator, discriminator2, discriminator3)
                    gen_loss = gen_loss1 + gen_loss2 + gen_loss3
                    # 计算判别器loss
                    disc_loss1 = LD1(wave_batch, noise, k, generator, discriminator)
                    disc_loss2 = LD(wave_batch, noise, k, generator, discriminator1, discriminator4)
                    disc_loss3 = LD(wave_batch, noise, k, generator, discriminator2, discriminator3)
                    disc_loss = disc_loss1 + disc_loss2 + disc_loss3
                # 传入loss和模型参数，计算生成器的权值调整
                gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
                # 传入loss和模型参数，计算判别器的权值调整
                gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
                # 生成器的权值调整
                generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
                # 判别器的权值调整
                discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
                #更新k
                k = tf.compat.v1.assign(k, tf.clip_by_value(k + lambda_k1 * balance, 0, 1))
        elif 500<epoch<=6000:
            for wave_batch in dataset:
                # 生成一个批次的随机数，这个随机数用于模型训练
                noise = tf.random.normal([batch_size, noise_dim_0])
                #输出goal全局得分
                goal,balance = global_pro(wave_batch,noise,gamma)
                #输出goal
                print('全局得分：%s'%goal)
                print('k:%s'%k)
                goal = float(goal)
                # print(goal)
                goal_container.append(goal)
                # 固定写法，使用tf.GradientTape()来计算梯度
                with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                    # 计算生成器loss
                    gen_loss1 = LG1(noise, generator, discriminator)
                    gen_loss2 = LG(noise, generator, discriminator1, discriminator4)
                    gen_loss3 = LG(noise, generator, discriminator2, discriminator3)
                    gen_loss = gen_loss1 + gen_loss2 + gen_loss3
                    # 计算判别器loss
                    disc_loss1 = LD1(wave_batch, noise, k, generator, discriminator)
                    disc_loss2 = LD(wave_batch, noise, k, generator, discriminator1, discriminator4)
                    disc_loss3 = LD(wave_batch, noise, k, generator, discriminator2, discriminator3)
                    disc_loss = disc_loss1 + disc_loss2 + disc_loss3
                # 传入loss和模型参数，计算生成器的权值调整
                gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
                # 传入loss和模型参数，计算判别器的权值调整
                gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
                # 生成器的权值调整
                generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
                # 判别器的权值调整
                discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
                #更新k
                k = tf.compat.v1.assign(k, tf.clip_by_value(k + lambda_k2 * balance, 0, 1))  
        with summary_K.as_default():
            tf.summary.scalar('k',k,step=epoch)
        with summary_LOSS.as_default():
            tf.summary.scalar('gen_loss',gen_loss,step=epoch)
            tf.summary.scalar('dis_loss',disc_loss,step=epoch)
        # 显示和保存图片
 
        print('第%s次训练'%epoch)
        print('xxxxxxxxxxxxxxxxxxxxxxxxx')

In [ ]:
test_input = tf.random.normal([120, noise_dim_0])

# # 需要将数据平滑，再进行下一步迭代
# b = savgol_filter(b, 41, 3, mode = "nearest")
# b = np.array(b)
# b = b.reshape(2500,1)

turning_point(test_input[0:1], generator)

In [ ]:
# 模型训练 
#记录运行时间
start = time.perf_counter()
train_0(train_dataset, epochs)
end = time.perf_counter()
print('Running time: %s Seconds'%(end-start))

In [ ]:
noise_test = tf.random.normal([600, noise_dim_0])
outPut = generator(noise_test)

In [ ]:
PlotSpectrum(outPut,P)

In [ ]:
generator.save('MSG_generatorLast.h5')
discriminator.save('MSG_discriminatorLast.h5')